In [ ]:
# -*- coding: utf-8 -*-
# Import các thư viện cần thiết
import re
import os
from slugify import slugify # Thư viện để tạo slug thân thiện với URL/tên file

# --- Cấu hình ---
input_file = 'output/kinh-tang-chi/vi-tmc/kinhtangchibo.md' # Tên file markdown nguồn

output_dir = 'output/kinh-tang-chi/vi-tmc/'       # Thư mục để lưu các file đã chia
encoding = 'utf-8'               # Encoding của file
number_padding = 3               # Số ký tự cho phần số (vd: 3 -> 001, 024)

# --- Logic xử lý ---

# Tạo thư mục output nếu chưa tồn tại
os.makedirs(output_dir, exist_ok=True)
print(f"Thư mục output: '{output_dir}'")

# Biến để lưu trữ file đang ghi và nội dung
current_file_handle = None
current_filename = None
file_count = 0

# Biểu thức chính quy để tìm dòng tiêu đề bài kinh
# ####\s+(\d+)\.\s+(.*)
# - ^####: Bắt đầu bằng ####
# - \s+: Theo sau bởi một hoặc nhiều khoảng trắng
# - (\d+): Nhóm 1: Một hoặc nhiều chữ số (số thứ tự bài kinh)
# - \.: Dấu chấm literal
# - \s+: Theo sau bởi một hoặc nhiều khoảng trắng
# - (.*): Nhóm 2: Bất kỳ ký tự nào cho đến cuối dòng (tên bài kinh)
pattern = re.compile(r'^####\s+(\d+)\.\s+(.*)', re.IGNORECASE)

# kinh tang chi - theo Phẩm - tieng viet
pattern = re.compile(r'^\*\*([IVXLCDM]+)\.\s+(.*?\bPhẩm\b.*?)\*\*')

try:
    with open(input_file, 'r', encoding=encoding) as infile:
        print(f"Đang đọc file: '{input_file}'...")
        for line in infile:
            match = pattern.match(line)
            if match:
                # Nếu tìm thấy dòng tiêu đề mới
                file_count += 1

                # Đóng file cũ nếu đang mở
                if current_file_handle:
                    current_file_handle.close()
                    print(f" -> Đã ghi xong: '{current_filename}'")

                # Lấy thông tin từ kết quả khớp regex
                kinh_num_str = match.group(1)
                kinh_title_raw = match.group(2).strip()

                # Tạo tên file slug
                padded_num = kinh_num_str.zfill(number_padding)
                slug_title = slugify(kinh_title_raw)
                filename = f"{padded_num}-{slug_title}.md"
                filepath = os.path.join(output_dir, filename)

                # Mở file mới để ghi
                print(f"\n{file_count}. Tìm thấy bài kinh: '{kinh_title_raw}' - Đang tạo file: '{filename}'...", end='')
                current_filename = filename
                current_file_handle = open(filepath, 'w', encoding=encoding)

            # Ghi dòng hiện tại vào file đang mở (nếu có)
            if current_file_handle:
                current_file_handle.write(line)

    # Đóng file cuối cùng sau khi kết thúc vòng lặp
    if current_file_handle:
        current_file_handle.close()
        print(f" -> Đã ghi xong: '{current_filename}'")

    print(f"\nHoàn tất! Đã chia thành công {file_count} file trong thư mục '{output_dir}'.")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{input_file}'. Vui lòng kiểm tra lại tên file và đường dẫn.")
except Exception as e:
    print(f"Đã xảy ra lỗi không mong muốn: {e}")
    # Đảm bảo đóng file nếu có lỗi xảy ra giữa chừng
    if current_file_handle and not current_file_handle.closed:
        current_file_handle.close()
        print(f"Đã đóng file '{current_filename}' do lỗi.")

In [ ]:
# split kinh tang chi tieng anh

import re
import os
from slugify import slugify # Thư viện để tạo slug thân thiện với URL/tên file

# --- Cấu hình ---
# Thay đổi input_file và output_dir cho phù hợp với file tiếng Anh
input_file = '../.docsource/kinh-tang-chi/sujato/5.md' # Tên file markdown nguồn
output_dir = '../.docsource/kinh-tang-chi/sujato/5/'       # Thư mục để lưu các file đã chia
encoding = 'utf-8'                # Encoding của file
number_padding_book = 2           # Số ký tự cho phần số Chương (vd: 2 -> 01)
number_padding_chapter = 3        # Số ký tự cho phần số Phẩm (vd: 3 -> 001, 010)

# --- Logic xử lý ---

# Tạo thư mục output nếu chưa tồn tại
os.makedirs(output_dir, exist_ok=True)
print(f"Thư mục output: '{output_dir}'")

# Biến để lưu trữ trạng thái
current_file_handle = None
current_filename = None
chapter_book_num = 10 # chương bắt đầu
chapter_num_in_book = 0
file_count = 0 # Tổng số file (phẩm) đã tạo

# Biểu thức chính quy
# Tìm dòng bắt đầu Chương (# The Book of the ...)
book_pattern = re.compile(r'^#\s+The Book of the\s+(.*)', re.IGNORECASE) # chia theo chương
# Tìm dòng bắt đầu Phẩm (# The Chapter on ...)
# chapter_pattern = re.compile(r'^#\s+.*Chapter|\w+ on\s+(.*)', re.IGNORECASE) # chia theo phẩm
# chapter_pattern = re.compile(r'^#\s+(?:(?:The|\w+)\s+Chapter(?:s?))\s+(.*)', re.IGNORECASE) # chia theo phẩm
chapter_pattern = re.compile(r'^##\s+(.*\sChapter(?:s?).*)', re.IGNORECASE)

# chapter_pattern = re.compile(r'^##\s+(.*)', re.IGNORECASE)
try:
    with open(input_file, 'r', encoding=encoding) as infile:
        print(f"Đang đọc file: '{input_file}'...")
        for line in infile:
            book_match = book_pattern.match(line)
            chapter_match = chapter_pattern.match(line)

            if book_match:
                # Nếu tìm thấy dòng bắt đầu Chương mới
                chapter_book_num += 1
                chapter_num_in_book = 0 # Reset số thứ tự phẩm khi sang chương mới
                # Không cần đóng file ở đây, chỉ đánh dấu bắt đầu chương mới
                print(f"\n--- Tìm thấy Chương {chapter_book_num}: '{book_match.group(1).strip()}' ---")

            elif chapter_match:
                # Nếu tìm thấy dòng bắt đầu Phẩm mới
                file_count += 1
                chapter_num_in_book += 1

                # Đóng file cũ (của phẩm trước đó) nếu đang mở
                if current_file_handle:
                    current_file_handle.close()
                    print(f" -> Đã ghi xong: '{current_filename}'")

                # Lấy thông tin từ kết quả khớp regex
                chapter_title_raw = chapter_match.group(1).strip()

                # Tạo tên file slug
                padded_book_num = str(chapter_book_num).zfill(number_padding_book)
                padded_chapter_num = str(chapter_num_in_book).zfill(number_padding_chapter)
                slug_title = slugify(chapter_title_raw)
                filename = f"{padded_book_num}-{padded_chapter_num}-{slug_title}.md"
                filepath = os.path.join(output_dir, filename)

                # Mở file mới để ghi
                print(f"\n{file_count}. (Chương {padded_book_num}, Phẩm {padded_chapter_num}): '{chapter_title_raw}' - Đang tạo file: '{filename}'...", end='')
                current_filename = filename
                current_file_handle = open(filepath, 'w', encoding=encoding)

            # Ghi dòng hiện tại vào file đang mở (nếu có)
            # File chỉ được mở khi gặp dòng '# The Chapter on...' đầu tiên
            if current_file_handle:
                current_file_handle.write(line)

    # Đóng file cuối cùng (của phẩm cuối cùng) sau khi kết thúc vòng lặp
    if current_file_handle:
        current_file_handle.close()
        print(f" -> Đã ghi xong: '{current_filename}'")

    print(f"\nHoàn tất! Đã chia thành công {file_count} file (phẩm) trong thư mục '{output_dir}'.")
    print("!!!Tiep theo tạo tiêu đề H1!!!")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{input_file}'. Vui lòng kiểm tra lại tên file và đường dẫn.")
except Exception as e:
    print(f"Đã xảy ra lỗi không mong muốn: {e}")
    # Đảm bảo đóng file nếu có lỗi xảy ra giữa chừng
    if current_file_handle and not current_file_handle.closed:
        current_file_handle.close()
        print(f"Đã đóng file '{current_filename}' do lỗi.")

In [ ]:
# đọc theo từng chương trước(1/2)
# kinh tang chi tieng anh theo Bodhi, vì orc nên hơi khó
# -*- coding: utf-8 -*-
# Import các thư viện cần thiết
import re
import os
from slugify import slugify # Thư viện để tạo slug thân thiện với URL/tên file

# --- Cấu hình ---
input_file = 'output/kinh-tang-chi/tangchi_NikayaBodhi2012.md' # Tên file markdown nguồn (Cần cập nhật nếu file nguồn khác)
output_dir = 'output/kinh-tang-chi/bodhi/' # Thư mục để lưu các file đã chia (Đã thay đổi thư mục con)
encoding = 'utf-8'               # Encoding của file
number_padding = 2               # Số ký tự cho phần số (vd: 2 -> 01, 11)

# Từ điển ánh xạ chữ số sang số nguyên
# Cần bổ sung nếu có nhiều hơn 11 quyển sách
number_word_map = {
    "ones": 1,
    "twos": 2,
    "threes": 3,
    "fours": 4,
    "fives": 5,
    "sixes": 6,
    "sevens": 7,
    "eights": 8,
    "nines": 9,
    "tens": 10,
    "elevens": 11,
    # Thêm các ánh xạ khác nếu cần, ví dụ: "twelves": 12
}

# --- Logic xử lý ---

# Tạo thư mục output nếu chưa tồn tại
os.makedirs(output_dir, exist_ok=True)
print(f"Thư mục output: '{output_dir}'")

# Biến để lưu trữ file đang ghi và nội dung
current_file_handle = None
current_filename = None
file_count = 0

# Biểu thức chính quy để tìm dòng tiêu đề quyển sách
# ^#: Bắt đầu bằng #
# \s+: Theo sau bởi một hoặc nhiều khoảng trắng
# (The Book of the\s+(\w+)): Nhóm 1: Toàn bộ tiêu đề "The Book of the ..."
#   \s+: Khoảng trắng sau "the"
#   (\w+): Nhóm 2: Từ chỉ số (ví dụ: "Ones", "Twos")
pattern = re.compile(r'^#\s+(The Book of the\s+(\w+))', re.IGNORECASE)
pattern = re.compile(r"^## [IVXLCDM]\. .*$")
try:
    with open(input_file, 'r', encoding=encoding) as infile:
        print(f"Đang đọc file: '{input_file}'...")
        for line in infile:
            match = pattern.match(line)
            if match:
                # Nếu tìm thấy dòng tiêu đề mới
                file_count += 1 # Tăng bộ đếm sách (không phải bài kinh)

                # Đóng file cũ nếu đang mở
                if current_file_handle:
                    current_file_handle.close()
                    print(f" -> Đã ghi xong: '{current_filename}'")

                # Lấy thông tin từ kết quả khớp regex
                book_title_raw = match.group(1).strip() # Ví dụ: "The Book of the Ones"
                number_word = match.group(2).strip().lower() # Ví dụ: "ones"

                # Chuyển đổi từ chữ sang số
                chapter_book_num = number_word_map.get(number_word)

                if chapter_book_num is None:
                    print(f"\nCẢNH BÁO: Không tìm thấy số cho từ '{number_word}' trong dòng: '{line.strip()}'. Bỏ qua phần này.")
                    # Không mở file mới, nội dung tiếp theo sẽ bị bỏ qua
                    # hoặc ghi vào file trước đó nếu muốn.
                    # Để đơn giản, ta sẽ không ghi gì cho đến khi gặp tiêu đề hợp lệ tiếp theo.
                    current_file_handle = None # Đảm bảo không ghi vào file cũ nữa
                    current_filename = None
                    continue # Chuyển sang dòng tiếp theo

                # Tạo tên file slug
                padded_num = str(chapter_book_num).zfill(number_padding) # Ví dụ: "01", "11"
                slug_title = slugify(book_title_raw)             # Ví dụ: "the-book-of-the-ones"
                filename = f"{padded_num}-{slug_title}.md"
                filepath = os.path.join(output_dir, filename)

                # Mở file mới để ghi
                print(f"\n{file_count}. Tìm thấy sách: '{book_title_raw}' - Đang tạo file: '{filename}'...", end='')
                current_filename = filename
                current_file_handle = open(filepath, 'w', encoding=encoding)

            # Ghi dòng hiện tại vào file đang mở (nếu có)
            # Chỉ ghi nếu đã tìm thấy ít nhất một tiêu đề sách hợp lệ
            if current_file_handle:
                current_file_handle.write(line)

    # Đóng file cuối cùng sau khi kết thúc vòng lặp
    if current_file_handle:
        current_file_handle.close()
        print(f" -> Đã ghi xong: '{current_filename}'")

    print(f"\nHoàn tất! Đã chia thành công {file_count} file trong thư mục '{output_dir}'.")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{input_file}'. Vui lòng kiểm tra lại tên file và đường dẫn.")
except Exception as e:
    print(f"Đã xảy ra lỗi không mong muốn: {e}")
    # Đảm bảo đóng file nếu có lỗi xảy ra giữa chừng
    if current_file_handle and not current_file_handle.closed:
        current_file_handle.close()
        print(f"Đã đóng file '{current_filename}' do lỗi.")

In [ ]:
# đọc theo từng phẩm sau khi đọc chương (2/2)
# Chương trình chia file markdown theo tiêu đề ## <la mã>. <văn bản>
# Tên file output: <parameter_number>-<so_la_ma>-<slug_van_ban>.md
# -*- coding: utf-8 -*-
# Import các thư viện cần thiết
import re
import os
from slugify import slugify # Thư viện để tạo slug thân thiện với URL/tên file

# --- Cấu hình ---

input_file = 'output/kinh-tang-chi/bodhi/02-the-book-of-the-twos.md' # Tên file markdown nguồn (Cần cập nhật)
output_dir = 'output/kinh-tang-chi/bodhi/pham/' # Thư mục để lưu các file đã chia (Đã thay đổi để tránh ghi đè lên file cũ)
encoding = 'utf-8'               # Encoding của file

# Cấu hình cho tên file output
# Đây là số parameter cố định sẽ đứng đầu tên file
parameter_number_for_filename = 2 # Đặt số parameter bạn muốn (ví dụ: 1 cho bộ đầu tiên)
# Số ký tự cho phần số parameter (ví dụ: 2 -> 01, 10)
parameter_number_padding = 2

# --- Logic xử lý ---

# Tạo thư mục output nếu chưa tồn tại
os.makedirs(output_dir, exist_ok=True)
print(f"Thư mục output: '{output_dir}'")

# Biến để lưu trữ file đang ghi và nội dung
current_file_handle = None
current_filename = None
file_count = 0 # Đếm số file/tiêu đề đã xử lý

# Biểu thức chính quy để tìm dòng tiêu đề: ## <la mã>. <văn bản> [^...] (tùy chọn)
# ^##\s+: Bắt đầu bằng ## theo sau bởi một hoặc nhiều khoảng trắng
# ([IVXLCDM]+): Nhóm 1 - Bắt số La Mã (I, V, X, L, C, D, M), một hoặc nhiều ký tự
# \s*\.\s+: Theo sau bởi 0 hoặc nhiều khoảng trắng, dấu chấm, 1 hoặc nhiều khoảng trắng
# (.*?): Nhóm 2 - Bắt bất kỳ ký tự nào (không tham lam), đây là phần văn bản cho slug
# \s*(\[\^.*?\])?$: Theo sau bởi 0 hoặc nhiều khoảng trắng, và tùy chọn nhóm 3 là [^...]
# re.IGNORECASE: Bỏ qua hoa thường khi khớp (cho số La Mã) - có thể bỏ nếu chỉ muốn khớp chữ hoa
pattern = re.compile(r'^##\s+([IVXLCDM]+)\s*\.\s+(.*?)\s*(\[\^.*?\])?$', re.IGNORECASE)

roman_to_arabic_map = {
    "i": 1, "ii": 2, "iii": 3, "iv": 4, "v": 5, "vi": 6, "vii": 7, "viii": 8, "ix": 9, "x": 10,
    "xi": 11, "xii": 12, "xiii": 13, "xiv": 14, "xv": 15, "xvi": 16, "xvii": 17, "xviii": 18, "xix": 19, "xx": 20,
    "xxi": 21, "xxii": 22, "xxiii": 23, "xxiv": 24, "xxv": 25, "xxvi": 26, "xxvii": 27, "xxviii": 28, "xxix": 29, "xxx": 30
    # Thêm các ánh xạ khác nếu cần, ví dụ: "c": 100, "m": 1000 (cho các số La Mã lớn hơn)
}

try:
    with open(input_file, 'r', encoding=encoding) as infile:
        print(f"Đang đọc file: '{input_file}'...")
        for line in infile:
            match = pattern.match(line)
            if match:
                # Nếu tìm thấy dòng tiêu đề mới
                file_count += 1 # Tăng bộ đếm file/tiêu đề

                # Đóng file cũ nếu đang mở
                if current_file_handle:
                    current_file_handle.close()
                    print(f" -> Đã ghi xong: '{current_filename}'")

                # Lấy thông tin từ kết quả khớp regex
                roman_numeral_str = match.group(1).strip() # Ví dụ: "X"
                text_part_for_slug = match.group(2).strip() # Ví dụ: "INTERNAL"

                # --- Chuyển đổi số La Mã sang số Arabic ---
                roman_numeral_lower = roman_numeral_str.lower()
                arabic_num = roman_to_arabic_map.get(roman_numeral_lower)


                # Tạo tên file slug theo định dạng <parameter_number>-<so_la_ma>-<slug>.md
                padded_param_num = str(parameter_number_for_filename).zfill(parameter_number_padding) # Ví dụ: "01"
                padded_arabic_num = str(arabic_num).zfill(2)
                slug_text = slugify(text_part_for_slug) if text_part_for_slug else "untitled" # Tạo slug, dùng "untitled" nếu phần văn bản rỗng

                # Chuyển số La Mã sang chữ thường cho tên file cho đồng nhất (tùy chọn)
                roman_numeral_for_filename = roman_numeral_str.lower()

                # filename = f"{padded_param_num}-{roman_numeral_for_filename}-{slug_text}.md"
                filename = f"{padded_param_num}-{padded_arabic_num}-{slug_text}.md"
                filepath = os.path.join(output_dir, filename)

                # Mở file mới để ghi
                print(f"\n{file_count}. Tìm thấy tiêu đề: '{line.strip()}' - Đang tạo file: '{filename}'...", end='')
                current_filename = filename
                # Sử dụng 'w' để ghi đè nếu file đã tồn tại, 'a' để nối thêm
                current_file_handle = open(filepath, 'w', encoding=encoding)

            # Ghi dòng hiện tại vào file đang mở (nếu có)
            # Chỉ ghi nếu đã tìm thấy ít nhất một tiêu đề hợp lệ và mở file
            if current_file_handle:
                current_file_handle.write(line)

    # Đóng file cuối cùng sau khi kết thúc vòng lặp
    if current_file_handle:
        current_file_handle.close()
        print(f" -> Đã ghi xong: '{current_filename}'")

    print(f"\nHoàn tất! Đã chia thành công {file_count} file trong thư mục '{output_dir}'.")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{input_file}'. Vui lòng kiểm tra lại tên file và đường dẫn.")
except Exception as e:
    print(f"Đã xảy ra lỗi không mong muốn: {e}")
    # Đảm bảo đóng file nếu có lỗi xảy ra giữa chừng
    if current_file_handle and not current_file_handle.closed:
        current_file_handle.close()
        print(f"Đã đóng file '{current_filename}' do lỗi.")

In [7]:
# -*- coding: utf-8 -*-
# kinh truong bo
import re
import os
from slugify import slugify # Thư viện để tạo slug thân thiện với URL/tên file

input_file = '../.docsource/kinhtuongung/tuongungbokinh.md' # Tên file markdown nguồn
output_dir = '../.docsource/kinhtuongung/tmc/'
encoding = 'utf-8'               # Encoding của file
number_padding = 2               # Số ký tự cho phần số (vd: 2 -> 01, 09, 10)

# --- Logic xử lý ---

# Tạo thư mục output nếu chưa tồn tại
os.makedirs(output_dir, exist_ok=True)
print(f"Thư mục output: '{output_dir}'")

# Biến để lưu trữ file đang ghi và nội dung
current_file_handle = None
current_filename = None
file_count = 0

# Biểu thức chính quy mới để tìm dòng tiêu đề bài kinh theo mẫu:
# ## DN <number> name of sutta
# - ^##: Bắt đầu bằng '##'
# - \s+: Theo sau bởi một hoặc nhiều khoảng trắng
# - DN: Chuỗi ký tự 'DN'
# - \s+: Theo sau bởi một hoặc nhiều khoảng trắng
# - (\d+): Nhóm 1: Một hoặc nhiều chữ số (số thứ tự bài kinh)
# - \s+: Theo sau bởi một hoặc nhiều khoảng trắng
# - (.*): Nhóm 2: Bất kỳ ký tự nào cho đến cuối dòng (tên bài kinh đầy đủ)
pattern = re.compile(r'^#\s+DN\s+(\d+)\s+(.*)', re.IGNORECASE) # Sử dụng re.IGNORECASE để chấp nhận 'dn' hoặc 'DN'

# kinh tuong ung tmc
pattern = re.compile(r'^#\s+(\d+)\.\s+(.*)', re.IGNORECASE) # # 2. TƯƠNG ƯNG THIÊN TỬ

try:
    with open(input_file, 'r', encoding=encoding) as infile:
        print(f"Đang đọc file: '{input_file}'...")
        for line in infile:
            match = pattern.match(line)
            if match:
                # Nếu tìm thấy dòng tiêu đề mới
                file_count += 1

                # Đóng file cũ nếu đang mở
                if current_file_handle:
                    current_file_handle.close()
                    print(f" -> Đã ghi xong: '{current_filename}'")

                # Lấy thông tin từ kết quả khớp regex
                kinh_num_str = match.group(1) # Lấy số bài kinh (ví dụ: "2")
                kinh_title_raw = match.group(2).strip() # Lấy tên bài kinh đầy đủ (ví dụ: "The Fruits of the Ascetic Life *Sāmaññaphalasutta*")

                # Tạo tên file slug theo định dạng <number>-name of sutta.md
                # Đảm bảo số có 2 chữ số (vd: "1" thành "01")
                padded_num = kinh_num_str.zfill(number_padding)

                # Tạo slug từ số và tiêu đề thô để đảm bảo uniqueness và dễ đọc
                # Ví dụ: "02-the-fruits-of-the-ascetic-life-samannaphalasutta"
                slug_for_filename = slugify(f"{padded_num}-{kinh_title_raw}")
                filename = f"{slug_for_filename}.md"

                filepath = os.path.join(output_dir, filename)

                # Mở file mới để ghi
                print(f"\n{file_count}. Tìm thấy bài kinh: '{kinh_title_raw}' (DN {kinh_num_str}) - Đang tạo file: '{filename}'...", end='')
                current_filename = filename
                current_file_handle = open(filepath, 'w', encoding=encoding)

            # Ghi dòng hiện tại vào file đang mở (nếu có)
            if current_file_handle:
                current_file_handle.write(line)

    # Đóng file cuối cùng sau khi kết thúc vòng lặp
    if current_file_handle:
        current_file_handle.close()
        print(f" -> Đã ghi xong: '{current_filename}'")

    print(f"\nHoàn tất! Đã chia thành công {file_count} file trong thư mục '{output_dir}'.")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{input_file}'. Vui lòng kiểm tra lại tên file và đường dẫn.")
except Exception as e:
    print(f"Đã xảy ra lỗi không mong muốn: {e}")
    # Đảm bảo đóng file nếu có lỗi xảy ra giữa chừng
    if current_file_handle and not current_file_handle.closed:
        current_file_handle.close()
        print(f"Đã đóng file '{current_filename}' do lỗi.")

Thư mục output: '../.docsource/kinhtuongung/tmc/'
Đang đọc file: '../.docsource/kinhtuongung/tuongungbokinh.md'...

1. Tìm thấy bài kinh: 'TƯƠNG ƯNG CHƯ THIÊN' (DN 1) - Đang tạo file: '01-tuong-ung-chu-thien.md'... -> Đã ghi xong: '01-tuong-ung-chu-thien.md'

2. Tìm thấy bài kinh: 'TƯƠNG ƯNG THIÊN TỬ' (DN 2) - Đang tạo file: '02-tuong-ung-thien-tu.md'... -> Đã ghi xong: '02-tuong-ung-thien-tu.md'

3. Tìm thấy bài kinh: 'TƯƠNG ƯNG KOSALA' (DN 3) - Đang tạo file: '03-tuong-ung-kosala.md'... -> Đã ghi xong: '03-tuong-ung-kosala.md'

4. Tìm thấy bài kinh: 'TƯƠNG ƯNG ÁC MA' (DN 4) - Đang tạo file: '04-tuong-ung-ac-ma.md'... -> Đã ghi xong: '04-tuong-ung-ac-ma.md'

5. Tìm thấy bài kinh: 'TƯƠNG ƯNG TỶ KHEO NI' (DN 5) - Đang tạo file: '05-tuong-ung-ty-kheo-ni.md'... -> Đã ghi xong: '05-tuong-ung-ty-kheo-ni.md'

6. Tìm thấy bài kinh: 'TƯƠNG ƯNG PHẠM THIÊN' (DN 6) - Đang tạo file: '06-tuong-ung-pham-thien.md'... -> Đã ghi xong: '06-tuong-ung-pham-thien.md'

7. Tìm thấy bài kinh: 'TƯƠNG ƯNG BÀ LA

In [2]:
# -*- coding: utf-8 -*-
# kinh truong bo khong dung DN <number>
import re
import os
from slugify import slugify # Thư viện để tạo slug thân thiện với URL/tên file

# --- Cấu hình ---


input_file = '../.docsource/kinhtuongung/sujato/5.md' # Tên file markdown nguồn
input_file = '../.scripts/output/jill-brain/jill-brain.md'
output_dir = '../.scripts/output/jill-brain/1/'
encoding = 'utf-8'               # Encoding của file
number_padding = 2               # Số ký tự cho phần số (vd: 2 -> 01, 09, 10)
init_suttra = 1                  # Số bắt đầu cho bài kinh đầu tiên


# input_file = '../.docsource/kinhtuongung/sujato/0/03-the-linked-discourses-the-blueprint-for-buddhist-philosophy.md' # Tên file markdown nguồn
# output_dir = '../.docsource/kinhtuongung/sujato/0/'
# encoding = 'utf-8'               # Encoding của file
# number_padding = 2               # Số ký tự cho phần số (vd: 2 -> 01, 09, 10)
# init_suttra = 55                  # Số bắt đầu cho bài kinh đầu tiên

# --- Logic xử lý ---

# Tạo thư mục output nếu chưa tồn tại
os.makedirs(output_dir, exist_ok=True)
print(f"Thư mục output: '{output_dir}'")

# Biến để lưu trữ file đang ghi và nội dung
current_file_handle = None
current_filename = None
file_count = 0  # Đếm số lượng file đã được tạo
sutta_number = init_suttra # Số bài kinh hiện tại, sẽ được dùng để đặt tên file và tăng dần

# Biểu thức chính quy mới để tìm dòng tiêu đề bài kinh theo mẫu:
# ## <name of sutta>
# - ^##: Bắt đầu bằng '##'
# - \s+: Theo sau bởi một hoặc nhiều khoảng trắng
# - (.*): Nhóm 1: Bất kỳ ký tự nào cho đến cuối dòng (tên bài kinh đầy đủ)
pattern = re.compile(r'^#\s+(.*)', re.IGNORECASE)
# pattern = re.compile(r'^#\s+DN \d+ (.*)', re.IGNORECASE)

try:
    with open(input_file, 'r', encoding=encoding) as infile:
        print(f"Đang đọc file: '{input_file}'...")
        for line in infile:
            match = pattern.match(line)
            if match:
                # Nếu tìm thấy dòng tiêu đề mới
                file_count += 1 # Tăng số lượng file đã xử lý

                # Đóng file cũ nếu đang mở
                if current_file_handle:
                    current_file_handle.close()
                    print(f" -> Đã ghi xong: '{current_filename}'")

                # Lấy tên bài kinh đầy đủ từ kết quả khớp regex
                kinh_title_raw = match.group(1).strip() # Lấy tên bài kinh đầy đủ

                # Tạo tên file slug
                # Đảm bảo số có đủ chữ số (vd: "1" thành "01")
                padded_num = str(sutta_number).zfill(number_padding)

                # Tạo slug từ số và tiêu đề thô để đảm bảo uniqueness và dễ đọc
                # Ví dụ: "01-a-readers-guide-to-the-pali-suttas"
                slug_for_filename = slugify(f"{padded_num}-{kinh_title_raw}")
                filename = f"{slug_for_filename}.md"

                filepath = os.path.join(output_dir, filename)

                # Tăng số bài kinh cho lần tiếp theo
                sutta_number += 1

                # Mở file mới để ghi
                print(f"\n{file_count}. Tìm thấy tiêu đề: '{kinh_title_raw}' (File số: {padded_num}) - Đang tạo file: '{filename}'...", end='')
                current_filename = filename
                current_file_handle = open(filepath, 'w', encoding=encoding)

            # Ghi dòng hiện tại vào file đang mở (nếu có)
            if current_file_handle:
                current_file_handle.write(line)

    # Đóng file cuối cùng sau khi kết thúc vòng lặp
    if current_file_handle:
        current_file_handle.close()
        print(f" -> Đã ghi xong: '{current_filename}'")

    print(f"\nHoàn tất! Đã chia thành công {file_count} file trong thư mục '{output_dir}'.")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{input_file}'. Vui lòng kiểm tra lại tên file và đường dẫn.")
except Exception as e:
    print(f"Đã xảy ra lỗi không mong muốn: {e}")
    # Đảm bảo đóng file nếu có lỗi xảy ra giữa chừng
    if current_file_handle and not current_file_handle.closed:
        current_file_handle.close()
        print(f"Đã đóng file '{current_filename}' do lỗi.")

Thư mục output: '../.scripts/output/jill-brain/1/'
Đang đọc file: '../.scripts/output/jill-brain/jill-brain.md'...

1. Tìm thấy tiêu đề: 'PREFACE' (File số: 01) - Đang tạo file: '01-preface.md'... -> Đã ghi xong: '01-preface.md'

2. Tìm thấy tiêu đề: 'MYSTORY AND OUR BRAIN' (File số: 02) - Đang tạo file: '02-mystory-and-our-brain.md'... -> Đã ghi xong: '02-mystory-and-our-brain.md'

3. Tìm thấy tiêu đề: 'BRAIN ANATOMY AND PERSONALITY' (File số: 03) - Đang tạo file: '03-brain-anatomy-and-personality.md'... -> Đã ghi xong: '03-brain-anatomy-and-personality.md'

4. Tìm thấy tiêu đề: 'OUR BRAIN’S TEAM : THE FOUR CHARACTERS' (File số: 04) - Đang tạo file: '04-our-brains-team-the-four-characters.md'... -> Đã ghi xong: '04-our-brains-team-the-four-characters.md'

5. Tìm thấy tiêu đề: 'CHARACTER 1 LEFT BRAIN THINKING' (File số: 05) - Đang tạo file: '05-character-1-left-brain-thinking.md'... -> Đã ghi xong: '05-character-1-left-brain-thinking.md'

6. Tìm thấy tiêu đề: 'CHARACTER 2 LEFT BRAIN EM